---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import pickle
import warnings
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
#     import matplotlib.pyplot as plt
    
#     g_num = 0
#     for graph in P1_Graphs:
#         print ('Graph', g_num)
#         print ('average shortest: ', nx.average_shortest_path_length(graph))
#         print ('average clustering: ',nx.average_clustering(graph))
#         print ('transitivity', nx.transitivity(graph))
#         degrees = graph.degree()
#         degree_values = sorted(set(degrees.values()))
#         histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(graph))for i in degree_values]
#         plt.bar(degree_values, histogram)
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of Nodes')
#         plt.show()
#         print ('')
#         g_num+=1
    return ['PA','SW_L', 'SW_L', 'PA', 'SW_H' ]

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [105]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    graph = G.nodes(data = True)
#     df = pd.DataFrame(graph)
    
    data={}
    data['node']=[x[0] for x in graph]
    data['Dept'] = [x[1]['Department'] for x in graph]
    data['Mgt Sal'] = [x[1]['ManagementSalary'] for x in graph]
    df = pd.DataFrame(data)
    degCent = nx.degree_centrality(G)
    df['degCent'] = df['node'].map(degCent)
    closeCent = nx.closeness_centrality(G)
    df['closeCent'] = df['node'].map(closeCent)
    between = nx.betweenness_centrality(G)
    df['between'] = df['node'].map(between)
    p_rank = nx.pagerank(G)
    df['p_rank'] = df['node'].map(p_rank)
    df['Dept'] = df['Dept'].astype(str)

#     print ('section 1')
    ##Drop node and dept columns
    df.drop(['Dept', 'node'], axis=1, inplace = True)
    ## pull out Mgt Sal NaN
    nans = df[df['Mgt Sal'].isnull()]
    nans.drop(['Mgt Sal'], axis=1, inplace = True)
    known = df[df['Mgt Sal'].notnull()]
    ##Convert Mgt Sal to int
    known['Mgt Sal'] = known['Mgt Sal'].astype(int)
   
#     print ('section 2')
    y = known['Mgt Sal']
    X = known.drop(['Mgt Sal'], axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    clf = GradientBoostingClassifier(learning_rate = .1, max_depth = 2, random_state = 0)
    clf.fit(X_train, y_train)
    ans = clf.predict_proba(nans)
    answer = pd.Series(ans[:,1], nans.index)

    return answer


# salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [127]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


,AA
0,
"(0, 2)",1.257626
"(0, 3)",0.605728
"(0, 4)",0.605728
"(0, 7)",0.938816
"(0, 8)",0.224500


In [117]:

future_connections.head()

,Future Connection,CN,PR,JC,RA,AA,CNSH,RASH,IC,MGR CNSH,MGR RASH
"(0, 2)",0.0,1.945910,8.338306,0.044784,0.053863,1.257626,1.945910,0.0,0.0,1.945910,0.000000
"(0, 3)",0.0,1.386294,8.047190,0.026907,0.021162,0.605728,1.386294,0.0,0.0,1.386294,0.000000
"(0, 4)",0.0,1.386294,8.348775,0.021979,0.021162,0.605728,1.386294,0.0,0.0,1.386294,0.000000
"(0, 7)",0.0,1.609438,8.061171,0.035718,0.059841,0.938816,1.609438,0.0,0.0,1.791759,0.013793
"(0, 8)",0.0,0.693147,7.395722,0.012739,0.011561,0.224500,0.693147,0.0,0.0,0.693147,0.000000


In [124]:
# rates = [ .2,]
rates = [.2]
depths = [1,2,3,4]
# depths = [1]
n_estimators = [100, 200, 300, 500, 1000]

for rate in rates:
    for depth in depths:

#         predicted = clf.predict_proba(X_test)
#         feature_importances = pd.DataFrame(clf.feature_importances_,
#                        index = X_train.columns,
#                         columns=['importance']).sort_values('importance',                                                                 
#                                                             ascending=False)
# # predicted
#         print ('rate:', rate,  'depth', depth,  'score', round(roc_auc_score(y_test, predicted[:,1]),3))
#         print (feature_importances)
#         print (" ")
answer

(0, 9)          0.028271
(0, 19)         0.058393
(0, 20)         0.179561
(0, 35)         0.015800
(0, 38)         0.015298
(0, 42)         0.823515
(0, 43)         0.013994
(0, 50)         0.014991
(0, 53)         0.574969
(0, 54)         0.038512
(0, 62)         0.507812
(0, 63)         0.058393
(0, 69)         0.028271
(0, 72)         0.013547
(0, 83)         0.161126
(0, 90)         0.028271
(0, 91)         0.015800
(0, 95)         0.079892
(0, 100)        0.028271
(0, 106)        0.979157
(0, 108)        0.015800
(0, 109)        0.013547
(0, 110)        0.013547
(0, 118)        0.015298
(0, 122)        0.028271
(0, 131)        0.038512
(0, 133)        0.697330
(0, 140)        0.059829
(0, 149)        0.101282
(0, 151)        0.015298
                  ...   
(988, 989)      0.013547
(988, 996)      0.013547
(988, 997)      0.062445
(988, 1000)     0.013547
(988, 1002)     0.013547
(989, 994)      0.013547
(989, 996)      0.013547
(989, 1003)     0.013547
(989, 1004)     0.013547


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [129]:
def new_connections_predictions():
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    
    cn = [((e[0],e[1]),  len(list(nx.common_neighbors(G, e[0],e[1])))) for e in nx.non_edges(G)]
    cn_df = pd.DataFrame.from_dict(cn)
    cn_df.set_index(0, inplace = True)
    cn_df.rename(columns = {1:'CN'}, inplace = True)
    cn_df['CN'] = np.log(cn_df['CN']+1)
    
    L = list(nx.preferential_attachment(G))
    l_fix = []
    for l in L:
        l_fix.append(((l[0],l[1]),l[2]))

    l_df = pd.DataFrame.from_dict(l_fix)
    l_df.set_index(0, inplace = True)
    l_df.rename(columns = {1:'PR'}, inplace = True)
    l_df['PR'] = np.log(l_df['PR']+1)
    
    
    jc = list(nx.jaccard_coefficient(G))
    jc_fix = []
    for i in jc:
        jc_fix.append(((i[0], i[1]),i[2]))
    jc_df = pd.DataFrame.from_dict(jc_fix)
    jc_df.set_index(0, inplace = True)
    jc_df.rename(columns = {1:'JC'}, inplace = True)
    jc_df['JC'] = np.log(jc_df['JC']+1)

    ra = list(nx.resource_allocation_index(G))
    ra_fix = []
    for r in ra:
        ra_fix.append(((r[0], r[1]),r[2]))
    ra_df = pd.DataFrame.from_dict(ra_fix)
    ra_df.set_index(0, inplace = True)
    ra_df.rename(columns = {1:'RA'}, inplace = True)
    ra_df['RA'] = np.log(ra_df['RA']+1)

    aa = list(nx.adamic_adar_index(G))
    aa_fix = []
    for a in aa:
        aa_fix.append(((a[0], a[1]),a[2]))
    aa_df = pd.DataFrame.from_dict(aa_fix)
    aa_df.set_index(0, inplace = True)
    aa_df.rename(columns = {1:'AA'}, inplace = True)

    for n in G.node:
        G.node[n]['community'] = str(G.node[n]['Department'])
    cnsh = list(nx.cn_soundarajan_hopcroft(G))
    cnsh_fix = []
    for c in cnsh:
        cnsh_fix.append(((c[0], c[1]),c[2]))
    cnsh_df = pd.DataFrame.from_dict(cnsh_fix)
    cnsh_df.set_index(0, inplace = True)
    cnsh_df.rename(columns = {1:'CNSH'}, inplace = True)
    cnsh_df['CNSH'] = np.log(cnsh_df['CNSH']+1)
    
    rash = list(nx.ra_index_soundarajan_hopcroft(G))
    rash_fix = []
    for r in rash:
        rash_fix.append(((r[0], r[1]), r[2]))
    rash_df = pd.DataFrame.from_dict(rash_fix)
    rash_df.set_index(0, inplace = True)
    rash_df.rename(columns = {1:'RASH'}, inplace = True)
    rash_df['RASH'] = np.log(rash_df['RASH']+1)
    
    ic = list(nx.within_inter_cluster(G))
    ic_fix = []
    for r in ic:
        ic_fix.append(((r[0], r[1]), r[2]))
    ic_df = pd.DataFrame.from_dict(ic_fix)
    ic_df.set_index(0, inplace = True)
    ic_df.rename(columns = {1:'IC'}, inplace = True)
    ic_df['IC'] = np.log(ic_df['IC']+1 )
    
    for n in G.node:
        G.node[n]['community'] = str(G.node[n]['ManagementSalary'])
    mgr_cnsh = list(nx.cn_soundarajan_hopcroft(G))
    mgr_cnsh_fix = []
    for c in mgr_cnsh:
        mgr_cnsh_fix.append(((c[0], c[1]),c[2]))
    mgr_cnsh_df = pd.DataFrame.from_dict(mgr_cnsh_fix)
    mgr_cnsh_df.set_index(0, inplace = True)
    mgr_cnsh_df.rename(columns = {1:'MGR CNSH'}, inplace = True)
    mgr_cnsh_df['MGR CNSH'] = np.log(mgr_cnsh_df['MGR CNSH']+1 )
    
    mgr_rash = list(nx.ra_index_soundarajan_hopcroft(G))
    mgr_rash_fix = []
    for c in mgr_rash:
        mgr_rash_fix.append(((c[0], c[1]),c[2]))
    mgr_rash_df = pd.DataFrame.from_dict(mgr_rash_fix)
    mgr_rash_df.set_index(0, inplace = True)
    mgr_rash_df.rename(columns = {1:'MGR RASH'}, inplace = True)
    mgr_rash_df['MGR RASH'] = np.log(mgr_rash_df['MGR RASH'] +1)
    
    future_connections.sort_index(inplace = True)
    future_connections = future_connections.join(cn_df)
    future_connections = future_connections.join(l_df)
    future_connections = future_connections.join(jc_df)
    future_connections = future_connections.join(ra_df)
    future_connections = future_connections.join(aa_df)
    future_connections = future_connections.join(cnsh_df)
    future_connections = future_connections.join(rash_df)
    future_connections = future_connections.join(ic_df)
    future_connections = future_connections.join(mgr_cnsh_df)
    future_connections = future_connections.join(mgr_rash_df)

    nans = future_connections[future_connections['Future Connection'].isnull()] 
    nans.drop(['Future Connection'], axis=1, inplace = True)
    known = future_connections[future_connections['Future Connection'].notnull()]
    y = known['Future Connection']
    X = known.drop(['Future Connection'], axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    clf = GradientBoostingClassifier(learning_rate = .2, max_depth = 1, random_state = 0)
    clf.fit(X_train, y_train)
    ans = clf.predict_proba(nans)
    answer = pd.Series(ans[:,1], nans.index)
    
    return answer

new_connections_predictions()

(0, 9)          0.028271
(0, 19)         0.058393
(0, 20)         0.179561
(0, 35)         0.015800
(0, 38)         0.015298
(0, 42)         0.823515
(0, 43)         0.013994
(0, 50)         0.014991
(0, 53)         0.574969
(0, 54)         0.038512
(0, 62)         0.507812
(0, 63)         0.058393
(0, 69)         0.028271
(0, 72)         0.013547
(0, 83)         0.161126
(0, 90)         0.028271
(0, 91)         0.015800
(0, 95)         0.079892
(0, 100)        0.028271
(0, 106)        0.979157
(0, 108)        0.015800
(0, 109)        0.013547
(0, 110)        0.013547
(0, 118)        0.015298
(0, 122)        0.028271
(0, 131)        0.038512
(0, 133)        0.697330
(0, 140)        0.059829
(0, 149)        0.101282
(0, 151)        0.015298
                  ...   
(988, 989)      0.013547
(988, 996)      0.013547
(988, 997)      0.062445
(988, 1000)     0.013547
(988, 1002)     0.013547
(989, 994)      0.013547
(989, 996)      0.013547
(989, 1003)     0.013547
(989, 1004)     0.013547
